In [3]:
from openai import OpenAI
import openai
import os
import ast
import re
import json
import markdown
from bs4 import BeautifulSoup
import pickle

import requests
from xml.etree import ElementTree
import ast

from collections import defaultdict
from difflib import SequenceMatcher
import numpy as np
from scipy.cluster.hierarchy import fcluster
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from linkml_runtime.loaders.yaml_loader import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView

In [4]:
openai.api_key = ""

In [5]:
# Initialize the OpenAI client
client = OpenAI(api_key=openai.api_key)#, base_url="https://api.cborg.lbl.gov")

In [6]:
# Define the function to query the OpenAI API
def query_openai(model, role, prompt):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": role, "content": prompt}
            ]
        )
        return response#.choices[0].message['content'] if response.choices else None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [7]:
#metadata-elements-comparison.txt


In [8]:
schema_path = "../src/data_sheets_schema/schema/data_sheets_schema.yaml"
# Load schema
#yaml_loader = YAMLLoader()
schema_view = SchemaView(schema_path)

print(f"Schema name: {schema_view.schema.name}")
classes = schema_view.all_classes()
print(f"Classes in the schema: {list(classes.keys())}")

INFO:root:Importing standards_organization_schema as standards_organization_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing standards_schema as standards_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing linkml:types as /Users/marcin/Documents/VIMSS/ontology/bridge2ai/data-sheets-schema/venv/lib/python3.10/site-packages/linkml_runtime/linkml_model/model/schema/types from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=None


Schema name: data-sheets-schema
Classes in the schema: ['NamedThing', 'AnatomicalEntity', 'Organization', 'OrganizationContainer', 'Information', 'FormatDialect', 'Person', 'DatasetProperty', 'DatasetCollection', 'Dataset', 'DataSubset', 'Software', 'Purpose', 'Task', 'AddressingGap', 'Creator', 'FundingMechanism', 'Grantor', 'Grant', 'Instance', 'SamplingStrategy', 'MissingInfo', 'Relationships', 'Splits', 'DataAnomaly', 'ExternalResource', 'Confidentiality', 'ContentWarning', 'Subpopulation', 'Deidentification', 'SensitiveElement', 'InstanceAcquisition', 'CollectionMechanism', 'DataCollector', 'CollectionTimeframe', 'EthicalReview', 'DirectCollection', 'CollectionNotification', 'CollectionConsent', 'ConsentRevocation', 'DataProtectionImpact', 'PreprocessingStrategy', 'CleaningStrategy', 'LabelingStrategy', 'RawData', 'ExistingUse', 'UseRepository', 'OtherTask', 'FutureUseImpact', 'DiscouragedUse', 'ThirdPartySharing', 'DistributionFormat', 'DistributionDate', 'LicenseAndUseTerms', 'I

In [9]:
schema_view

SchemaView(schema=SchemaDefinition({
  'name': 'data-sheets-schema',
  'description': 'A LinkML schema for Datasheets for Datasets.',
  'title': 'data-sheets-schema',
  'see_also': ['https://bridge2ai.github.io/data-sheets-schema'],
  'id': 'https://w3id.org/bridge2ai/data-sheets-schema',
  'imports': ['linkml:types', 'standards_schema', 'standards_organization_schema'],
  'license': 'MIT',
  'prefixes': {'biolink': Prefix({'prefix_prefix': 'biolink', 'prefix_reference': 'https://w3id.org/biolink/vocab/'}),
    'csvw': Prefix({'prefix_prefix': 'csvw', 'prefix_reference': 'http://www.w3.org/ns/csvw#'}),
    'data_sheets_schema': Prefix({
      'prefix_prefix': 'data_sheets_schema',
      'prefix_reference': 'https://w3id.org/bridge2ai/data-sheets-schema/'
    }),
    'datasets': Prefix({'prefix_prefix': 'datasets', 'prefix_reference': 'https://w3id.org/linkml/report'}),
    'dcat': Prefix({'prefix_prefix': 'dcat', 'prefix_reference': 'http://www.w3.org/ns/dcat#'}),
    'example': Prefix

In [10]:
# Get all classes
classes = schema_view.all_classes()
for class_name, class_def in classes.items():
    print(f"Class: {class_name}, Definition: {class_def}")

# Get all slots
slots = schema_view.all_slots()
for slot_name, slot_def in slots.items():
    print(f"Slot: {slot_name}, Definition: {slot_def}")

# Get all enums
enums = schema_view.all_enums()
for enum_name, enum_def in enums.items():
    print(f"Enum: {enum_name}, Definition: {enum_def}")


Class: NamedThing, Definition: ClassDefinition({
  'name': 'NamedThing',
  'description': 'A generic grouping for any identifiable entity',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'slots': ['id', 'category', 'name', 'description', 'subclass_of', 'related_to',
    'contributor_name', 'contributor_github_name', 'contributor_orcid',
    'contribution_date'],
  'class_uri': 'schema:Thing'
})
Class: AnatomicalEntity, Definition: ClassDefinition({
  'name': 'AnatomicalEntity',
  'id_prefixes': ['uberon'],
  'description': 'A subcellular location, cell type or gross anatomical part',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'exact_mappings': ['uberon:0001062', 'wikidata:Q4936952'],
  'narrow_mappings': ['ncit:C12219'],
  'is_a': 'NamedThing'
})
Class: Organization, Definition: ClassDefinition({
  'name': 'Organization',
  'description': ('Represents a group or organization related to or responsible for one or more '
     'Bridge2AI standards.

In [11]:
class_details = {}

for class_name, class_def in schema_view.all_classes().items():
    description = class_def.description if class_def.description else 'No description available'
    subsets = class_def.in_subset if class_def.in_subset else []
    attributes = {}
    for attr_name, attr_def in class_def.attributes.items():
        attributes[attr_name] = attr_def.description if attr_def.description else 'No description available'
    
    class_details[class_name] = {
        'subset': subsets,
        'attributes': attributes,
        'description': description
    }

# Print the resulting dictionary for verification
for class_name, details in class_details.items():
    print(f"Class: {class_name}")
    print(f"  Subsets: {details['subset']}")
    print(f"  Description: {details['description']}")
    print(f"  Attributes:")
    for attr_name, attr_desc in details['attributes'].items():
        print(f"    - {attr_name}: {attr_desc}")
    print()

Class: NamedThing
  Subsets: []
  Description: A generic grouping for any identifiable entity
  Attributes:

Class: AnatomicalEntity
  Subsets: []
  Description: A subcellular location, cell type or gross anatomical part
  Attributes:

Class: Organization
  Subsets: []
  Description: Represents a group or organization related to or responsible for one or more Bridge2AI standards.
  Attributes:

Class: OrganizationContainer
  Subsets: []
  Description: A container for Organizations.
  Attributes:

Class: Information
  Subsets: []
  Description: Grouping for datasets and data files
  Attributes:

Class: FormatDialect
  Subsets: []
  Description: Additional format information for a file
  Attributes:
    - comment_prefix: No description available
    - delimiter: No description available
    - double_quote: No description available
    - header: No description available
    - quote_char: No description available

Class: Person
  Subsets: []
  Description: An individual human being.
  Attr

In [12]:
# Prepare the class details as a formatted string for the prompt
class_details_str = "\n".join(
    f"Class name: {class_name}\n"
    f"  Subsets: {', '.join(details['subset'])}\n"
    f"  Description: {details['description']}\n"
    f"  Attributes:\n" +
    "".join(f"    {attr_name}: {attr_desc}\n" for attr_name, attr_desc in details['attributes'].items())
    for class_name, details in class_details.items()
)


In [13]:
# Read the Markdown file content
with open("./healthsheet.md", 'r', encoding='utf-8') as file:
    markdown_content = file.read()

# Convert the Markdown content to HTML
html_content = markdown.markdown(markdown_content)

print(html_content)

<h1>Healthsheet for AI-READI Dataset</h1>
<p>This healthsheet provides information about the AI-READI dataset in line with a previously established template (Rostamzadeh et al., https://arxiv.org/abs/2202.13028) aimed at facilitating consistent and transparent documentation regarding the creation, use, and maintenance of healthcare datasets for machine learning.</p>
<h2>General information</h2>
<ol>
<li>Provide a 2 sentence summary of this dataset.</li>
</ol>
<p>The Artificial Intelligence Ready and Equitable Atlas for Diabetes Insights (AI-READI) is a dataset consisting of data collected from individuals with and without "Type 2 Diabetes Mellitus (T2DM)" and harmonized across 3 data collection sites. The composition of the dataset was designed with future studies using AI/Machine Learning in mind. This included recruitment sampling procedures aimed at achieving approximately equal distribution of participants across sex, race, and diabetes severity, as well as the design of a data acq

In [14]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Find all <li> tags and print them
list_items = soup.find_all('li')
for item in list_items:
    print("* "+item.get_text())

* Provide a 2 sentence summary of this dataset.
* Has the dataset been audited before? If yes, by whom and what are the results?
* Does the dataset get released as static versions or is it dynamically updated?
* Is this datasheet created for the original version of the dataset? If not, which version of the dataset is this datasheet for?
* Are there any datasheets created for any versions of this dataset?
* Does the current version/subversion of the dataset come with predefined task(s), labels, and recommended data splits (e.g., for training, development/validation, testing)? If yes, please provide a high-level description of the introduced tasks, data splits, and labeling, and explain the rationale behind them. Please provide the related links and references. If not, is there any resource (website, portal, etc.) to keep track of all defined tasks and/or associated label definitions? (please note that more detailed questions w.r.t labeling is provided in further sections)
* If the datas

In [29]:
llm_prompt_template = """
You are given a Healthsheet schema entry and a dictionary of class details from another schema. Your tasks are as follows:

1. **Identify Relevant Classes**: Based on the schema Class, Subsets, Description, and Attributes, find the most relevant Class for each entry.
   - If multiple classes are potential matches, list all.
   - If no reasonable match is found, report "No match" in the 'Class' field.
2. **Note Subsets and Specific Fields**: 
   - Even when there's no match to a Class, record matches to more general Subsets or specific fields within other Classes.
   - Use the format `ClassName.Attribute` for attributes for Classes.
3. **Describe Match Type**: Use the Closeness field to describe the match type: "exact", "close", or "partial".

Format the output as below, ensuring that matches to different Classes for the same entry will have separate output sections in this format.
Do not add any other text to the output and make sure to adhere to this output format including naming, structuring, and ordering of fields.

**Healthsheet entry:** "<entry_text>"
  - **Closeness:** "<closeness>"
  - **Class:** "<class_name>"
  - **Subset:** "<subset>"
  - **Attributes:** "<field1>, <field2>"

---

**Example:**

**Healthsheet entry:** "Has the dataset been audited before? If yes, by whom and what were the results?"
  - **Closeness:** "close"
  - **Class:** "EthicalReview"
  - **Subset:** "Collection"
  - **Attributes:** "EthicalReview.description"

Healthsheet entry: "Has the dataset been audited before? If yes, by whom and what were the results?"
  - **Closeness:** "close"
  - **Class:** "Dataset"
  - **Subset:** ""
  - **Attributes:** "Dataset.ethical_reviews"

Healthsheet entry: "Has the dataset been audited before? If yes, by whom and what were the results?"
  - **Closeness:** "close"
  - **Class:** "DataProtectionImpact"
  - **Subset:** "Collection"
  - **Attributes:** "DataProtectionImpact.description"

---

Here is the Healthsheet schema entry:

{list_item}

Here are the class details from the other schema:

{class_details}

"""

In [ ]:
llm_results = []

# Debugging function to display raw representation of entries
def show_raw_string(entry):
    return repr(entry)

# Function to strip HTML/Markdown tags
def strip_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Loop over Healthsheets entries, validate that it starts with <li>, fill out the prompt template, and submit to the LLM
for entry in list_items:
    # Ensure entry is not None and is a string after stripping whitespace
    raw_entry = str(entry).strip() if entry is not None else entry
    
    # Print raw and processed entries for debugging
    print(f"Raw Entry: {show_raw_string(entry)} | Processed Entry: {raw_entry}")
    
    if raw_entry is None or not isinstance(raw_entry, str) or not raw_entry.startswith("<li>"):
        print(f"Invalid entry: {raw_entry}. It does not start with <li>.")
        continue  # Skip to the next entry

    # Strip markdown/HTML tags
    stripped_entry = strip_tags(raw_entry)
    
    # Print raw and stripped entries for debugging
    print(f"Stripped Entry: {stripped_entry}")
    
    prompt = llm_prompt_template.format(
        list_item=f"- {stripped_entry}",
        class_details=class_details_str
    )
    
    # Call to LLM (Simulated in this example)
    # result = query_openai("gpt-4o", "user", prompt)  # "openai/gpt-4o"
    result = f"Simulated result for prompt: {prompt}"  # Simulated result for demonstration

    print(prompt)  # For debugging
    llm_results.append(result)
    
# Output the results
for result in llm_results:
    print(result)
    print("------")

In [46]:
# Function to safely extract data between two markers
def extract_between(content, start, end):
    try:
        return content.split(start)[1].split(end)[0].strip()
    except IndexError:
        return "Not Found"

# Parse the LLM responses and extract the required data
parsed_data = []

for response in llm_results:
    try:
        response_content = response.choices[0].message.content
        # Split the response content into individual sections
        entries = response_content.split("\n\n---\n\n")
        
        for entry in entries:
            entry_text = extract_between(entry, '**Healthsheet entry:** "', '"')
            closeness = extract_between(entry, '- **Closeness:** "', '"\n')
            class_name = extract_between(entry, '- **Class:** "', '"\n')
            subset = extract_between(entry, '- **Subset:** "', '"\n')
            most_relevant_fields = extract_between(entry, '- **Attributes:** "', '"\n')
            
            parsed_data.append({
                "Healthsheet entry": entry_text,
                "Closeness": closeness,
                "Class": class_name,
                "Subset": subset,
                "Attributes": most_relevant_fields
            })

    except Exception as e:
        print(f"An error occurred while parsing response: {e}")

# Create a DataFrame with the extracted data
healthsheet_to_d4d_mapping_df = pd.DataFrame(parsed_data, columns=["Healthsheet entry", "Closeness", "Class", "Subset", "Attributes"])

# Save the DataFrame to a TSV file
healthsheet_to_d4d_mapping_df.to_csv('healthsheet_into_datasheet_llm_results_v3.tsv', sep='\t', index=False)


In [31]:
# Save the object to a file
with open('data_vs_health_sheet_alignment_raw_v3.pkl', 'wb') as file:
    pickle.dump(llm_results, file)

In [ ]:
with open('data_vs_health_sheet_alignment_raw_v3.pkl', 'rb') as file:
    llm_results = pickle.load(file)

In [39]:
for entry in list_items:
    if raw_entry.startswith("<li>"):
        print(entry)

<li>Provide a 2 sentence summary of this dataset.</li>
<li>Has the dataset been audited before? If yes, by whom and what are the results?</li>
<li>Does the dataset get released as static versions or is it dynamically updated?</li>
<li>Is this datasheet created for the original version of the dataset? If not, which version of the dataset is this datasheet for?</li>
<li>Are there any datasheets created for any versions of this dataset?</li>
<li>Does the current version/subversion of the dataset come with predefined task(s), labels, and recommended data splits (e.g., for training, development/validation, testing)? If yes, please provide a high-level description of the introduced tasks, data splits, and labeling, and explain the rationale behind them. Please provide the related links and references. If not, is there any resource (website, portal, etc.) to keep track of all defined tasks and/or associated label definitions? (please note that more detailed questions w.r.t labeling is provide

In [47]:
healthsheet_to_d4d_mapping_df

,Healthsheet entry,Closeness,Class,Subset,Attributes
0,Provide a 2 sentence summary of this dataset.,exact,Dataset,,"Dataset.purposes"""
1,Provide a 2 sentence summary of this dataset.,close,Information,,""""
2,Provide a 2 sentence summary of this dataset.,close,NamedThing,,
3,"Has the dataset been audited before? If yes, b...",close,EthicalReview,Collection,"EthicalReview.description"""
4,"Has the dataset been audited before? If yes, b...",close,Dataset,Not Found,"Dataset.ethical_reviews"""
...,...,...,...,...,...
228,"Will the dataset be updated (e.g., to correct ...",partial,Maintainer,Maintenance,"Maintainer.description"""
229,Not Found,Not Found,Not Found,Not Found,Not Found
230,"If the dataset relates to people, are there ap...",exact,RetentionLimits,,description
231,Will older versions of the dataset continue to...,exact,VersionAccess,Maintenance,VersionAccess.description


In [48]:
df[df['Class'] != 'Not Found']

,Healthsheet entry,Closeness,Class,Subset,Attributes
0,Provide a 2 sentence summary of this dataset.,exact,Dataset,,"Dataset.purposes"""
1,Provide a 2 sentence summary of this dataset.,close,Information,,""""
2,Provide a 2 sentence summary of this dataset.,close,NamedThing,,
3,"Has the dataset been audited before? If yes, b...",close,EthicalReview,Collection,"EthicalReview.description"""
4,"Has the dataset been audited before? If yes, b...",close,Dataset,Not Found,"Dataset.ethical_reviews"""
...,...,...,...,...,...
227,"Will the dataset be updated (e.g., to correct ...",partial,Dataset,,"Dataset.updates"""
228,"Will the dataset be updated (e.g., to correct ...",partial,Maintainer,Maintenance,"Maintainer.description"""
230,"If the dataset relates to people, are there ap...",exact,RetentionLimits,,description
231,Will older versions of the dataset continue to...,exact,VersionAccess,Maintenance,VersionAccess.description


In [49]:
healthsheet_to_d4d_mapping_df[healthsheet_to_d4d_mapping_df['Class'] == 'Not Found']

,Healthsheet entry,Closeness,Class,Subset,Attributes
40,Not Found,Not Found,Not Found,Not Found,Not Found
72,Not Found,Not Found,Not Found,Not Found,Not Found
124,Not Found,Not Found,Not Found,Not Found,Not Found
150,Not Found,Not Found,Not Found,Not Found,Not Found
213,Not Found,Not Found,Not Found,Not Found,Not Found
222,Not Found,Not Found,Not Found,Not Found,Not Found
225,Not Found,Not Found,Not Found,Not Found,Not Found
229,Not Found,Not Found,Not Found,Not Found,Not Found
